<a href="https://colab.research.google.com/github/nicolay-r/THOR-ECAC/blob/master/THoR_Finetuning_SemEval2023_t3_1_public.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Switch to Python 3.8

In [ ]:
!sudo apt-get install python3.8 python3.8-distutils
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.8 1
!sudo update-alternatives --config python3
!python3 --version
!sudo apt install python3-pip

In [ ]:
%cd /content/
!rm -rf THOR-ECAC
!git clone --branch master https://github.com/nicolay-r/THOR-ECAC.git

We install the necessary project requirements

In [ ]:
%cd THOR-ECAC
!pip install -r requirements.txt

# Download necessary datasets

In [ ]:
%cd THOR-ECAC
!python download_data.py \
  --cause-test "https://www.dropbox.com/scl/fi/4b2ouqdhgifqy3pmopq08/cause-mult-test.csv?rlkey=tkw0p1e01vezrjbou6v7qh36a&dl=1" \
  --cause-train "https://www.dropbox.com/scl/fi/0tlkwbe5awcss2qmihglf/cause-mult-train.csv?rlkey=x9on1ogzn5kigx7c32waudi21&dl=1" \
  --cause-valid "https://www.dropbox.com/scl/fi/8zjng2uyghbkpbfcogj6o/cause-mult-valid.csv?rlkey=91dgg4ly7p23e3id2230lqsoi&dl=1" \
  --state-train "https://www.dropbox.com/scl/fi/0lokgaeo973wo82ig01hy/state-mult-train.csv?rlkey=tkt1oyo8kwgqs6gp79jn5vbh8&dl=1" \
  --state-valid "https://www.dropbox.com/scl/fi/eu4yuk8n61izygnfncnbo/state-mult-valid.csv?rlkey=tlg8rac4ofkbl9o4ipq6dtyos&dl=1"

# Prompt-Tuning Experiments


## Launching Stage 1

This is the pre-training stage for the LLM, aimed on understanding character state.

> **NOTE:** We use `thor_state` engine. Other engines are still available for experiments.

In [ ]:
!python main.py -r thor_state -es 4 -d state_se24 -bs 4

## Launching Stage 2

This is the second stage, aimed on LLM fine-tuning on emotion-cause pairs.

Let's consider that we fine-tune the following state, obtained from the stage 1:
*  /content/THOR-ECAC/data/save/base_state_se24_1.pth.tar

Using three different engines: `prompt_cause`, `thor_cause`, and `thor_cause_rr`.

In [ ]:
!python main.py -r prompt_cause -es 4 -d cause_se24 -bs 32 -lp "/content/THOR-ECAC/data/save/base_state_se24_1.pth.tar"

In [ ]:
!python main.py -r thor_cause -es 4 -d cause_se24 -bs 32 -lp "/content/THOR-ECAC/data/save/base_state_se24_1.pth.tar"

In [ ]:
!python main.py -r thor_cause_rr -es 4 -d cause_se24 -bs 32 -lp "/content/THOR-ECAC/data/save/base_state_se24_1.pth.tar"

# Infer results on the `test` set

In this section we infer the results on `test` set in order to later form the Codalab submission.

To infer the results on `test` set, we utilize `-i` parameter.

> **NOTE:** To load the fine-tuned version of the model for inferring, we can utilize the paramter `-li` instead, which is load by specific epoch index (which is `2` in our case). You can still use `-lp`, but `-li` is shorter.

In [ ]:
!python main.py -r prompt_cause -d cause_se24 -bs 64 -li 2 -i

In [ ]:
!python main.py -r thor_cause -d cause_se24 -bs 64 -li 2 -i

In [ ]:
!python main.py -r thor_cause_rr -d cause_se24 -bs 64 -li 2 -i